In [1]:
from pyspark import SparkContext
sc = SparkContext()

In [8]:
from collections import namedtuple
from operator import itemgetter

In [4]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.read_csv("dataset/ISTAT/occupazione.csv",sep='|')

,ITTER107,Territory,Territorio,TIPO_DATO_CENS_POP,Data Type,Tipo dato,SEXISTAT1,Gender,Sesso,ETA1,Age,Età,STATCIV2,Legal marital status,Stato civile,CITTADINANZA,Citizenship,Cittadinanza,ISO1,Geographical area of citizenship,Area geografica di cittadinanza,TITOLO_STUDIO,Educational attainment,Grado di istruzione,FORMAZIONE,Attendance to one or more vocational trainings,Frequenza di uno o più corsi di formazione professionale,CONDIZIONE_DICH,Current activity status,Condizione professionale o non professionale,STRUTTURE,Type of institutional household,Tipo di convivenza,TIME,Census year,Anno di Censimento,Value,Flags
0,ITG,Isole,Isole,RESPOP_AV,resident population (absolute values),popolazione residente (valori assoluti),1,males,maschi,Y_GE15,15 years and over,15 anni e più,99,total,totale,FRG,foreign,straniero-a,ALL,all items,tutte le voci,99,total,totale,ALL,all items,tutte le voci,1,employed person,occupato,ALL,all items,tutte le voci,2011,2011,2011,38799,NaN
1,ITG,Isole,Isole,RESPOP_AV,resident population (absolute values),popolazione residente (valori assoluti),2,females,femmine,Y_GE15,15 years and over,15 anni e più,99,total,totale,FRG,foreign,straniero-a,ALL,all items,tutte le voci,99,total,totale,ALL,all items,tutte le voci,1,employed person,occupato,ALL,all items,tutte le voci,2011,2011,2011,29975,NaN
2,ITG,Isole,Isole,RESPOP_AV,resident population (absolute values),popolazione residente (valori assoluti),9,total,totale,Y_GE15,15 years and over,15 anni e più,99,total,totale,FRG,foreign,straniero-a,ALL,all items,tutte le voci,99,total,totale,ALL,all items,tutte le voci,1,employed person,occupato,ALL,all items,tutte le voci,2011,2011,2011,68774,NaN
3,ITE4,Lazio,Lazio,RESPOP_AV,resident population (absolute values),popolazione residente (valori assoluti),1,males,maschi,Y_GE15,15 years and over,15 anni e più,99,total,totale,FRG,foreign,straniero-a,ALL,all items,tutte le voci,99,total,totale,ALL,all items,tutte le voci,1,employed person,occupato,ALL,all items,tutte le voci,2011,2011,2011,102743,NaN
4,ITE4,Lazio,Lazio,RESPOP_AV,resident population (absolute values),popolazione residente (valori assoluti),2,females,femmine,Y_GE15,15 years and over,15 anni e più,99,total,totale,FRG,foreign,straniero-a,ALL,all items,tutte le voci,99,total,totale,ALL,all items,tutte le voci,1,employed person,occupato,ALL,all items,tutte le voci,2011,2011,2011,102407,NaN
5,ITE4,Lazio,Lazio,RESPOP_AV,resident population (absolute values),popolazione residente (valori assoluti),9,total,totale,Y_GE15,15 years and over,15 anni e più,99,total,totale,FRG,foreign,straniero-a,ALL,all items,tutte le voci,99,total,totale,ALL,all items,tutte le voci,1,employed person,occupato,ALL,all items,tutte le voci,2011,2011,2011,205150,NaN
6,ITF2,Molise,Molise,RESPOP_AV,resident population (absolute values),popolazione residente (valori assoluti),1,males,maschi,Y_GE15,15 years and over,15 anni e più,99,total,totale,FRG,foreign,straniero-a,ALL,all items,tutte le voci,99,total,totale,ALL,all items,tutte le voci,1,employed person,occupato,ALL,all items,tutte le voci,2011,2011,2011,1770,NaN
7,ITF2,Molise,Molise,RESPOP_AV,resident population (absolute values),popolazione residente (valori assoluti),2,females,femmine,Y_GE15,15 years and over,15 anni e più,99,total,totale,FRG,foreign,straniero-a,ALL,all items,tutte le voci,99,total,totale,ALL,all items,tutte le voci,1,employed person,occupato,ALL,all items,tutte le voci,2011,2011,2011,1692,NaN
8,ITF2,Molise,Molise,RESPOP_AV,resident population (absolute values),popolazione residente (valori assoluti),9,total,totale,Y_GE15,15 years and over,15 anni e più,99,total,totale,FRG,foreign,straniero-a,ALL,all items,tutte le voci,99,total,totale,ALL,all items,tutte le voci,1,employed person,occupato,ALL,all items,tutte le voci,2011,2011,2011,3462,NaN
9,ITG,Isole,Isole,RESPOP_AV,resident population (absolute values),popolazione residente (valori assoluti),1,males,maschi,Y_GE15,15 years and over,15 anni e più,99,total,totale,FRG,foreign,

In [100]:
def parse_occupazione(sc,filepath):
    rawRdd = sc.textFile(filepath)
    header = rawRdd.first()

    # Togliamo l'header
    rawRdd = rawRdd.filter(lambda x: x != header)
    return rawRdd.map(lambda x: x.split("|")).filter(lambda x: 
          x[0].find("I") == -1 and 
          x[3] == '"RESPOP_AV"' and #RESIDENTI
          x[6] == '"9"' and #SESSO
          x[9] == '"Y_GE15"' and #ETA
          x[12] == '"99"' and #STATO CIVILE                                            
          x[15] == '"TOTAL"' and #CITTADINANZA
          x[18] == '"ALL"' and #ISO1
          x[21] == '"99"' and  #TITOLO_STUDIO           
          x[24] == '"ALL"' #FORMAZIONE
        )

rddOccupazione = parse_occupazione(sc,"dataset/ISTAT/occupazione.csv")

In [108]:
lmbd_f = lambda x: (int(x[0][1:-1]),int(x[36]))

rddForzaLavoro = rddOccupazione.filter(lambda x: x[27] == '"14"').map(lmbd_f)
rddTotale = rddOccupazione.filter(lambda x: x[27] == '"99"').map(lmbd_f)
rddOccupato = rddOccupazione.filter(lambda x: x[27] == '"1"').map(lmbd_f)

istat = namedtuple("forza_lavoro",["codice_istat","totale","occupato","forza_lavoro"])
rddFinal = rddTotale.join(rddOccupato).join(rddForzaLavoro).map(lambda x: 
         istat(
            codice_istat = x[0],
            totale = x[1][0][0],
            occupato = x[1][0][1],
            forza_lavoro = x[1][1])
          )

In [109]:
# 48017 -> Firenze
rddFinal.filter(lambda x: x[0] == 48017).take(1)

[forza_lavoro(codice_istat=48017, totale=315401, occupato=154105, forza_lavoro=165358)]

In [111]:
# % forza lavoro = forza lavoro / totale
# % occupazione = occupato / forza lavoro
istat_indici = namedtuple("istat",["codice_istat","forza_lavoro","occupazione"])
rddIndici = rddFinal.map(lambda x: istat_indici(
    codice_istat = x.codice_istat,
    forza_lavoro = x.forza_lavoro / x.totale,
    occupazione = x.occupato / x.forza_lavoro))
rddIndici.take(10)

[istat(codice_istat=23040, forza_lavoro=0.5705852417302799, occupazione=0.9414912593649661)]

In [112]:
collected_rdd = rddCoordinates.collect()
ipref_df = pd.DataFrame.from_records(ipref_df,columns = ipref._fields)